In [59]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle
import time

In [60]:

# Define the dataset class
class CarPriceDataset(Dataset):
    def __init__(self, data, labels=None, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        if self.transform:
            item = self.transform(item)
        if self.labels is not None:
            label = self.labels[idx]
            return item, label
        return item

# Define the neural network model with dropout and batch normalization for better accuracy
class CarPriceModel(nn.Module):
    def __init__(self, input_size, hidden_size1=128, hidden_size2=64, dropout_rate=0.3):
        super(CarPriceModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.BatchNorm1d(hidden_size1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size1, hidden_size2),
            nn.BatchNorm1d(hidden_size2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.layer3 = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


In [61]:

# Load and preprocess the data
data = pd.read_csv('Dataset/train.csv')
test_data = pd.read_csv('Dataset/test.csv')


In [62]:

# Handle missing values and encode categorical features
def preprocess_data(data, is_test=False):
    if not is_test:
        data = data.drop(columns=['id'], errors='ignore')
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le
    return data, label_encoders

data, label_encoders = preprocess_data(data)
test_data, _ = preprocess_data(test_data, is_test=True)


In [63]:

# Feature engineering: create new features from existing ones
data['age'] = 2023 - data['model_year']
test_data['age'] = 2023 - test_data['model_year']

# Extract features and target variable
X = data.drop(columns=['price'])
y = data['price'].values
X_test = test_data.drop(columns=['id'], errors='ignore')

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = CarPriceDataset(X_train, y_train)
val_dataset = CarPriceDataset(X_val, y_val)
test_dataset = CarPriceDataset(X_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, loss function, and optimizer
input_size = X.shape[1]
model = CarPriceModel(input_size=input_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [64]:

# Hyperparameter tuning
def hyperparameter_tuning(train_loader, val_loader):
    best_params = None
    best_loss = float('inf')

    for hidden_size1 in [64, 128, 256]:
        for hidden_size2 in [32, 64, 128]:
            for dropout_rate in [0.2, 0.3, 0.4]:
                model = CarPriceModel(input_size=input_size, hidden_size1=hidden_size1, hidden_size2=hidden_size2, dropout_rate=dropout_rate)
                optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
                train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
                val_loss, _ = evaluate_model(model, val_loader, criterion)
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_params = (hidden_size1, hidden_size2, dropout_rate)
    return best_params

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50):
    best_model = None
    best_loss = float('inf')
    train_times = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs.flatten(), targets.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_time = time.time() - start_time
        train_times.append(epoch_time)

        val_loss, val_rmse = evaluate_model(model, val_loader, criterion)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Val RMSE: {val_rmse}, Time: {epoch_time:.2f}s')

        # Save the best model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model.state_dict()

    avg_train_time = np.mean(train_times)
    print(f'Average training time per epoch: {avg_train_time:.2f}s')

    # Load the best model
    model.load_state_dict(best_model)

    # Save the model and the scaler
    with open('car_price_model.pkl', 'wb') as f:
        pickle.dump(model.state_dict(), f)
    with open('scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

# Evaluation function
def evaluate_model(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    all_targets = []
    all_outputs = []

    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs.float())
            loss = criterion(outputs.flatten(), targets.float())
            val_loss += loss.item()
            all_targets.extend(targets.numpy())
            all_outputs.extend(outputs.flatten().numpy())

    rmse = mean_squared_error(all_targets, all_outputs, squared=False)
    return val_loss / len(loader), rmse


In [65]:

# Perform hyperparameter tuning
best_params = hyperparameter_tuning(train_loader, val_loader)
print(f'Best parameters: {best_params}')

# Initialize the model with the best parameters
model = CarPriceModel(input_size=input_size, hidden_size1=best_params[0], hidden_size2=best_params[1], dropout_rate=best_params[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model with the best parameters
train_model(model, train_loader, val_loader, criterion, optimizer)

# Evaluate the model
val_loss, val_rmse = evaluate_model(model, val_loader, criterion)
print(f'Validation Loss: {val_loss}, Validation RMSE: {val_rmse}')


c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/10, Loss: 7434378528.424153, Val Loss: 4430661021.364706, Val RMSE: 66609.12295076357, Time: 2.40s


c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 2/10, Loss: 7425875490.3092785, Val Loss: 4418573976.847059, Val RMSE: 66518.31097920184, Time: 3.03s


c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 49/50, Loss: 5301431676.9838, Val Loss: 2309933946.5411763, Val RMSE: 48110.30080247598, Time: 3.16s


c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 50/50, Loss: 5305049070.185567, Val Loss: 2313815442.635294, Val RMSE: 48151.67766748918, Time: 3.42s
Average training time per epoch: 3.63s
Validation Loss: 2313815442.635294, Validation RMSE: 48151.67766748918


c:\Users\ravim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [66]:

# Predict on test data
def predict(model, loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in loader:
            outputs = model(inputs.float())
            predictions.extend(outputs.flatten().tolist())

    return predictions

# Load the best model and scaler
with open('car_price_model.pkl', 'rb') as f:
    model.load_state_dict(pickle.load(f))
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Make predictions on the test set
predictions = predict(model, test_loader)



In [67]:

# Create submission file
submission = pd.DataFrame({'id': test_data['id'], 'price': predictions})
submission.to_csv('submission.csv', index=False)
